In [1]:
!pip install torch

In [2]:
import torch
print("GPU доступен:", torch.cuda.is_available())


GPU доступен: True


In [3]:
!pip install evaluate

In [4]:
!pip install transformers accelerate


In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Загружаем модель и токенизатор
model_name = "EleutherAI/pythia-19m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name,torch_dtype=torch.float16,
                                             device_map="auto")

# Перемещаем модель на GPU (если доступен)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


d:\Documents\Python\hackathon_hse25\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 512)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-5): 6 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
          (dense): Linear(in_features=512, out_features=512, bias=True)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
          (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
          (act): GELUActivation()
        )
      )
    )
    (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise

In [6]:
import json
import pandas as pd
import ast  # Для корректного парсинга строковых списков
import re   # Для извлечения нужной части строки
from typing import List, Dict, Any

# --- Функции для парсинга контекста ---
def _parse_contexts(resources: str) -> List[List[str]]:
    """Парсинг контекстов с использованием регулярных выражений"""
    contexts = []
    pattern = re.compile(r"Document\(page_content='(.*?)', metadata=({.*?})\)", re.DOTALL)

    for match in re.finditer(pattern, resources):
        content, metadata_str = match.groups()
        try:
            metadata = ast.literal_eval(metadata_str)
            text = _clean_text(content)
            contexts.append(text)  # Добавляем только очищенный текст
        except Exception as e:
            print(f"Контекст не распарсился: {e}")

    return contexts  # Возвращаем список текстов

def _clean_text(text: str) -> str:
    """Очистка текста"""
    if not text:
        return ''
    return re.sub(r'\\[nrt]|[\n\r\t]+|\s+', ' ', text).strip()



In [7]:
import json
import pandas as pd
import ast  # Для корректного парсинга строковых списков
import re   # Для извлечения нужной части строки

# Загружаем данные из JSON
with open('./datasets/train_set.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Список для хранения обработанных данных
formatted_data = []

# Обрабатываем каждую запись
for item in data:
    question = item.get("Вопрос пользователя", "Нет вопроса")

    # Берем список контекстов из "Ресурсы для ответа"
    contexts = _parse_contexts(item.get("Ресурсы для ответа", ""))
    answer = item.get("Ответ AI")

    # Определяем победителя (чей ответ использовать)
    winner1 = item.get("Saiga")
    winner2 = item.get("Giga")
    winner = winner2 if item.get("Кто лучше?") == "Giga" else winner1

    # Преобразуем строку в список объектов, если это строка (иногда JSON неправильно парсится)
    # try:
    #     contexts_data = ast.literal_eval(raw_contexts) if isinstance(raw_contexts, str) else raw_contexts
    #     contexts = []

    #     for doc in contexts_data:
    #         if isinstance(doc, str):  # Если запись строковая, ищем нужную часть
    #             match = re.search(r"page_content='(.*?)', metadata=", doc, re.DOTALL)
    #             if match:
    #                 contexts.append(match.group(1))  # Извлекаем текст внутри page_content
    #         elif isinstance(doc, dict) and 'page_content' in doc:
    #             contexts.append(doc['page_content'])  # Обычный случай для словарей

    # except (SyntaxError, ValueError):
    #     contexts = []

    # Формируем финальную структуру
    formatted_data.append({
        "question": question,
        "answer": answer,
        "ground_truth": winner,  # В файле нет эталонных ответов, ставим заглушку
        "contexts": contexts
    })

# Создаем DataFrame
df = pd.DataFrame(formatted_data)

# Сохраняем в CSV и JSON
df.to_csv("./datasets/formatted_dataset.csv", index=False, encoding="utf-8")
df.to_json("./datasets/formatted_dataset.json", orient="records", force_ascii=False)

print("✅ Данные сохранены в formatted_dataset.csv и formatted_dataset.json!")


✅ Данные сохранены в formatted_dataset.csv и formatted_dataset.json!


In [8]:
import pandas as pd
import json

# Загружаем обработанные данные
df = pd.read_json("./datasets/formatted_dataset.json")

# Создаем словарь, где каждому ключу соответствует список значений
dataset_dict = {
    "question": df["question"].tolist(),       # Список всех вопросов
    "answer": df["answer"].tolist(),           # Список всех ответов модели
    "ground_truth": df["ground_truth"].tolist(),  # Список всех эталонных ответов
    "contexts": df["contexts"].tolist()  # Список списков контекстов
}

# Преобразуем словарь обратно в DataFrame (для сохранения)
final_df = pd.DataFrame.from_dict(dataset_dict)

# Сохраняем итоговые данные
with open("./datasets/final_dataset.json", "w", encoding="utf-8") as f:
    json.dump(dataset_dict, f, ensure_ascii=False, indent=4)

#final_df.to_csv("/mnt/data/final_dataset.csv", index=False, encoding="utf-8")

print("✅ Данные корректно преобразованы и сохранены в final_dataset.json и final_dataset.csv!")


✅ Данные корректно преобразованы и сохранены в final_dataset.json и final_dataset.csv!


In [9]:

import pandas as pd


dataset = pd.read_json("./datasets/final_dataset.json")


print(dataset.head())


                                            question  \
0  Для чего нужно подробное изучение науки, прове...   
1  Нужно ли подавать какие-то дополнительные доку...   
2             я 120см, смогу ли я попасть на военку?   
3  Нужно ли подавать какие-то дополнительные доку...   
4  Если во время прохождения теста в смарт лмс пр...   

                                              answer  \
0  Я могу частично ответить на данный вопрос: про...   
1  Дополнительные документы для продления социаль...   
2  Что-то в вашем вопросе меня смущает. Может, по...   
3  Дополнительные документы для продления социаль...   
4  Я не нашел конкретного ответа на ваш вопрос о ...   

                                        ground_truth  \
0  Я понял ваш вопрос. Вы хотите узнать, почему н...   
1  Для продления социальной стипендии необходимо ...   
2  Я понял ваш страх по поводу возможности поступ...   
3  Для продления социальной стипендии необходимо ...   
4  Я понял ваш вопрос о том, что случилось с т

In [10]:
!pip install tqdm

In [11]:
from tqdm import tqdm

In [12]:
!pip install evaluate

In [13]:
!pip install rouge_score


In [14]:
!pip install sacrebleu

In [15]:
!pip install bert_score

In [ ]:
import sys
sys.path.append('./datasets/')
from typing import List

import evaluate
import numpy as np
import pandas as pd
from tqdm import tqdm

rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")
chrf = evaluate.load("chrf")
bertscore = evaluate.load("bertscore")

# Теперь можно импортировать функцию
from metrics import ValidatorSimple

vs = ValidatorSimple(neural=True)
vs.validate_rag(dataset.head(300))

SyntaxError: incomplete input (308830075.py, line 19)

In [17]:
import matplotlib.pyplot as plt


In [18]:
save_results_to_json(vs,r"D:\Documents\Python\hackathon_hse25\prepocess_calculate\outpu.json")

NameError: name 'save_results_to_json' is not defined

In [ ]:
!pip install fastapi uvicorn

  Using cached fastapi-0.115.11-py3-none-any.whl.metadata (27 kB)
  Using cached uvicorn-0.34.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached starlette-0.46.0-py3-none-any.whl.metadata (6.2 kB)
  Using cached pydantic-2.10.6-py3-none-any.whl.metadata (30 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.27.2-cp312-cp312-win_amd64.whl.metadata (6.7 kB)
  Using cached anyio-4.8.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached fastapi-0.115.11-py3-none-any.whl (94 kB)
Using cached uvicorn-0.34.0-py3-none-any.whl (62 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 